</div>
<div align="left">
  <img src="img/abstract.png" width="400" alt="Funny little diagram">
  <p><em> Evolve nodes, evolve plans, and learn from the best performing ones.</em></p>
</div>
<div align="center">
</em></p>
</div>

#### Node Initialization (Refactoring ...)

In [1]:
from methods.llm import get_async_vllm_endpoint
import os 

# Unlimited LLM endpoints
endpoint_id = "vllm-4qqnnwfdggvjba"
api_key = "rpa_EPOJED42G59S80Y6SKMCOI330EQU4JPPMKV2UD2W7j0uku"
get_endpoint_response = get_async_vllm_endpoint(endpoint_id, api_key)

Could not load vllm class, check CUDA support and GPU RAM size


In [2]:
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.evolnode import EvolNode
from methods.llm import get_groq_response, get_claude_response

# Code + Compilor Task
mp = MetaPrompt("Search for age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.CODE)
# Prompt + LLM Task
# mp = MetaPrompt("Get the age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.PROMPT) # 

test_cases = [
    ({"name": "Dilireba"}, {"age": 32}),
    ({"name": "ChengXiao"}, {"age": 26})
]

test_inputs = [c[0] for c in test_cases]

node = EvolNode(mp, None, None, get_response=get_endpoint_response, test_cases=test_cases) # setting manual test cases

node.evolve("i1", replace=True, batch_size=40, num_runs=2, print_summary=True) # Scale up batch size


input_dict = {"name": "Dilireba"}
output_dict = node(input_dict, max_attempts=6) # Batch Inference with vLLM

# node.get_response = get_groq_response # fast sequential inference
# output_dict = node(input_dict, max_attempts=6, batch_inference=False)
 
print("Output dict: ", output_dict)

Processing LLM queries: 100%|██████████| 40/40 [01:16<00:00,  1.92s/it]


 :: Total time elapsed: 76.80s, 0 errors
An error occurred: No connection adapters were found for '"title": "Dilraba Dilmurat - Wikipedia",'
An error occurred: No connection adapters were found for '"title": "Cheng Xiao - Wikipedia",'


Processing LLM queries: 0it [00:00, ?it/s]


 :: Total time elapsed: 0.00s, 0 errors
🏆 Best Code Performance Summary 🏆
  ⚡ Structural fitness: 1.00
  🎯 Functional fitness: 1.00
  ⭐ Global fitness:     1.00
  🔄 Compiled solutions:        40
  ⏱️ Time breakdown:
     :: Query time: 12.95s
     :: Evolution time: 76.89s
     :: Evaluation time: 77.32s
     :: Total time: 167.16s


📊 Code 0: Fitness: 0.0%
--------------------------------------------------------------------------------
❌ Error Messages:
typing.Union cannot be used with isinstance()
typing.Union cannot be used with isinstance()


📊 Code 1: Fitness: 75.0%
--------------------------------------------------------------------------------
❌ Error Messages:
Input: {'name': 'ChengXiao'}, prediction is not aligned with expected output, Expected: {'age': 26} Predicted: {'age': -1}, Error message: 
Value mismatch for key age: -1 != 26




📊 Code 2: Fitness: 50.0%
--------------------------------------------------------------------------------
❌ Error Messages:
Input: {'name': 'D

</div>
<div align="center">
  <img src="img/Project-Nirvana-evolve.gif" width="500" alt="Fourier reconstruction convergence">
  <p><em> Evolve a population of nodes. </em></p>
</div>

In [4]:
# Population building phase ... 
from methods.llm import get_groq_response, get_claude_response
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.population import Evolution

mp = MetaPrompt("Get the age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.PROMPT) # 

test_cases = [
    ({"name": "Dilireba"}, {"age": 32}),
    ({"name": "ChengXiao"}, {"age": 26})
]

evo = Evolution(pop_size=20, meta_prompt=mp, get_response=get_endpoint_response, 
                test_cases=test_cases, max_attempts=3, num_eval_runs=2,
                load=True)

strategies = ["m2"] # ["i1", "i1", "m2", "e2"]
evo.get_offspring(strategies)

evo.chat("How effective is the current evolution strategy? What improvement has it made in terms of fitness, and in terms of the implementation?",
         get_claude_response) 

# code-based check 
print(evo.population_info)

Processing LLM queries: 100%|██████████| 20/20 [00:22<00:00,  1.11s/it]


 :: Total time elapsed: 22.27s, 0 errors


Processing LLM queries: 100%|██████████| 22/22 [00:37<00:00,  1.71s/it]


 :: Total time elapsed: 37.61s, 0 errors


Processing LLM queries: 0it [00:00, ?it/s]


 :: Total time elapsed: 0.00s, 0 errors
🏆 Best Code Performance Summary 🏆
  ⚡ Structural fitness: 0.33
  🎯 Functional fitness: 1.00
  ⭐ Global fitness:     0.67
  🔄 Compiled solutions:        17
  ⏱️ Time breakdown:
     :: Query time: 2.59s
     :: Evolution time: 22.32s
     :: Evaluation time: 68.43s
     :: Total time: 93.33s


📊 Code 0: Fitness: 8.3%
--------------------------------------------------------------------------------
❌ Error Messages:
Input: {'name': 'ChengXiao'}, prediction is not aligned with expected output, Expected: {'age': 26} Predicted: {'birth_date': {'answerBox': {'snippet': 'Cheng Xiao (Chinese: 程潇; Korean: 성소, born July 15, 1998) is a Chinese singer, dancer, and actress.', 'snippetHighlighted': ['July 15, 1998'], 'title': 'Cheng Xiao - Wikipedia', 'link': 'https://en.wikipedia.org/wiki/Cheng_Xiao'}, 'Search Result': [{'title': 'Cheng Xiao (WJSN) profile, age & facts (2024 updated) | kpopping', 'link': 'https://kpopping.com/profiles/idol/Cheng-Xiao', 'snippe


</div>
<div align="center">
  <img src="https://github.com/user-attachments/assets/af98faeb-66d6-4278-af86-67d668d1954e" width="900" alt="Fourier reconstruction convergence">
  <p><em> Plan, and evolve the plans. </em></p>
</div>


In [3]:
from methods.llm import get_claude_response, get_groq_response
from methods.diagram import visualize_plan_dict
from methods.meta_prompt import MetaPlan
from methods.evolnode import PlanNode


# Initialize PlanNode 
mp = MetaPlan("Get the age of celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"])
plan = PlanNode(mp, get_endpoint_response)

# i1 evolution of plan
plan_dicts, err_msg = plan.evolve_plan_dict(method="i1", batch_size=10) # Batch_size of 100 gives no slow-down

visualize_plan_dict(plan.plan_dict, plan.meta_prompt.task) # most simpliest plan

# Manual input on main-node test cases 
main_test_cases = [
    ({"name": "Dilireba"}, {"age": 32}),
    ({"name": "ChengXiao"}, {"age": 26})
]

is_success, err_msg = plan.spawn_test_cases(main_test_cases) #  pinned test cases generation
# plan.spawn_test_cases_majority(main_test_cases) # multi-agent test cases generation (need some benchmarking to compare quality)

 :: Evolving 10 plans in parallel...


Processing LLM queries: 100%|██████████| 10/10 [00:16<00:00,  1.65s/it]


 :: Total time elapsed: 16.47s, 0 errors
 :: Pseudo-code generated for each plan


Processing LLM queries: 100%|██████████| 10/10 [00:18<00:00,  1.89s/it]


 :: Total time elapsed: 18.89s, 0 errors
 :: Plan_dict generated for each plan


success: successfully compiled d2_output/plan_graph.d2 to d2_output/plan_graph.png in 190.064251ms
Processing LLM queries: 100%|██████████| 20/20 [00:20<00:00,  1.05s/it]

 :: Total time elapsed: 20.92s, 0 errors
Spawned 3 test cases for all sub-nodes


In [4]:
# plan.evolve_sub_nodes() # Completely stuck in the first call, debugging ... 

from methods.evolnode import EvolNode
from methods.meta_prompt import MetaPrompt, PromptMode

self = plan 

# 1. Did we skip existing nodes? Yes, those node has code & fitness and is skipped.
# 2. Did we make the name compatible with slightly off input?

for i, node_dict in enumerate(self.plan_dict["nodes"]):
    meta_prompt = MetaPrompt(
        task=node_dict["task"],
        func_name=node_dict["name"],
        inputs=node_dict["inputs"],
        outputs=node_dict["outputs"],
        input_types=node_dict["input_types"],
        output_types=node_dict["output_types"],
        mode=PromptMode((node_dict.get("mode", "code")).lower())
    )
    test_cases = self.test_cases_dict[node_dict["name"]]
    if "fitness" in node_dict and "code" in node_dict: 
        node = EvolNode(meta_prompt, node_dict["code"], node_dict["reasoning"], get_response=self.get_response, test_cases=test_cases, fitness=node_dict["fitness"])
    else:
        node = EvolNode(meta_prompt, None, None, get_response=self.get_response, test_cases=test_cases)
        print(f"🎲 :: Evolving {node.meta_prompt.func_name} ... ({i+1}/{len(self.plan_dict['nodes'])})")
        node.evolve("i1", replace=True, max_tries=2, num_runs=2, batch_size=20) # It's funny how 30+ sec could elapse before llm inference ... (collecting prompts ?? wtf is taking so long ??)
    self.nodes.append(node)

🎲 :: Evolving extract_birthdate ... (2/3)


Processing LLM queries: 100%|██████████| 20/20 [00:22<00:00,  1.11s/it]


 :: Total time elapsed: 22.28s, 0 errors


Processing LLM queries: 100%|██████████| 36/36 [00:15<00:00,  2.39it/s]


 :: Total time elapsed: 15.08s, 0 errors
🏆 Best Code Performance Summary 🏆
  ⚡ Structural fitness: 1.00
  🎯 Functional fitness: 1.00
  ⭐ Global fitness:     1.00
  🔄 Compiled solutions:        20
  ⏱️ Time breakdown:
     :: Query time: 2.98s
     :: Evolution time: 22.34s
     :: Evaluation time: 24.05s
     :: Total time: 49.37s


📊 Code 0: Fitness: 0.0%
--------------------------------------------------------------------------------
❌ Error Messages:
no such group
no such group


📊 Code 1: Fitness: 0.0%
--------------------------------------------------------------------------------
❌ Error Messages:
typing.Union cannot be used with isinstance()
typing.Union cannot be used with isinstance()


📊 Code 2: Fitness: 50.0%
--------------------------------------------------------------------------------
❌ Error Messages:
Input: {'search_result': 'Dilireba was born on June 3, 1992'}, prediction is not aligned with expected output, Expected: {'birthdate': 'June 3, 1992'} Predicted: {'birthda

Processing LLM queries: 100%|██████████| 20/20 [00:17<00:00,  1.13it/s]


 :: Total time elapsed: 17.68s, 0 errors
Error occurred during API request: Function execution timed out (> 3 seconds)


Processing LLM queries: 0it [00:00, ?it/s]

 :: Total time elapsed: 0.00s, 0 errors
🏆 Best Code Performance Summary 🏆
  ⚡ Structural fitness: 1.00
  🎯 Functional fitness: 0.50
  ⭐ Global fitness:     0.75
  🔄 Compiled solutions:        19
  ⏱️ Time breakdown:
     :: Query time: 2.90s
     :: Evolution time: 17.74s
     :: Evaluation time: 45.73s
     :: Total time: 66.37s


📊 Code 0: Fitness: 75.0%
--------------------------------------------------------------------------------
❌ Error Messages:
Input: {'birthdate': 'June 5, 1996'}, prediction is not aligned with expected output, Expected: {'age': 26} Predicted: {'age': 28}, Error message: 
Value mismatch for key age: 28 != 26




📊 Code 1: Fitness: 0.0%
--------------------------------------------------------------------------------
❌ Error Messages:
time data 'June 3, 1992' does not match format '%Y-%m-%d'
time data 'June 5, 1996' does not match format '%Y-%m-%d'


📊 Code 2: Fitness: 0.0%
--------------------------------------------------------------------------------
❌ Error